In [4]:
import os
import requests
import dotenv
import numpy as np
os.environ["MODELS_OUTPUT_DIR"] = "/kaggle/working/models"
os.environ["MODELS_PRETRAINED_DIR"]= "/kaggle/working/models/pretrained"
import langchain
from youtube_transcript_api import YouTubeTranscriptApi
import sentence_transformers
from yandex_chain import YandexLLM
from googleapiclient.discovery import build
from deepmultilingualpunctuation import PunctuationModel
from tqdm import tqdm
from youtube_transcript_api._errors import NoTranscriptFound,TranscriptsDisabled
import pandas as pd

from dotenv import load_dotenv
from src.data.transcripts_prep import LoadTranscripts
from src.data.finetuning_prep import FinetuningTemplateMaker
from src.data.rag_vectorbase_split import TextVBLoader

from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain import HuggingFacePipeline 
from typing import Any, List, Mapping, Optional
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import torch
import langchain



/Users/ernestgatiatullin/Desktop/github/PyProjects/my-projects/DivertitoGPT/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
load_dotenv()
YANDEX_FOLDER_ID = os.environ.get("YANDEX_FOLDER_ID")
YANDEX_API_KEY = os.environ.get("YANDEX_API_KEY")


In [6]:
ya_llm = YandexLLM(folder_id=YANDEX_FOLDER_ID, api_key=YANDEX_API_KEY)
# ya_llm("привет")

In [7]:
transcripts_path = "../data/interim/punctuated_trancripts.csv"
playlist_ids_dict = {
        "Лекции_успешные_свидания_2012": "PLrSae6j9lpRIJ7W250Eb1YXWCKcBX_Ztl",
        "Истории отношений": "PLrSae6j9lpRINAX7AV4OptFKsgB0xC5TY",
        "Мужские правила": "PLrSae6j9lpRJezIrfOSudu8Gme-VB2p2e"
    }
standalone_video_ids = {"Как забыть бывшую девушку?💔":"KXL57PM6a5s"}

if os.path.exists(transcripts_path):
    transcripts_df = pd.read_csv(transcripts_path)
    transcripts_list_punct = transcripts_df["transripts_with_punctuation"].values.tolist()
else:
    load_tscripts = LoadTranscripts(transcripts_path, playlist_ids_dict,standalone_video_ids)
    standalone_df = load_tscripts.get_standalone_video_ids()
    transcripts_df, transcripts_list_punct = load_tscripts()
    transcripts_df = pd.concat([transcripts_df, standalone_df])
transcripts_df.head()

,playlist,title,transripts_with_punctuation
0,Лекции_успешные_свидания_2012,Успешные свидания 2021. Вступление,"Всем привет С вами, Максим вертито. кстати, ка..."
1,Лекции_успешные_свидания_2012,Успешные свидания 2021. Начало свидания. Лекция 1,"бомбераскачки с первого раза погнали. Ну, вы п..."
2,Лекции_успешные_свидания_2012,Успешные свидания 2021. О чем говорить на свид...,Всем привет С вами Максим вертит и мы продолжа...
3,Лекции_успешные_свидания_2012,Успешные свидания 2021. Как флиртовать с девуш...,"Итак, всем привет С вами, Максим диверсита. Вы..."
4,Лекции_успешные_свидания_2012,Как правильно трогать девушку! Успешные свидан...,"Всем привет С вами, Максим дерти. я продолжаю ..."


# Make FineTuning file 

In [8]:
xlsx_path = "../data/interim/divertitogpt_finetuning_52exp.xlsx"
save_finetune_json = "../data/interim/divertitogpt_finetuning_52exp.json"
ftm = FinetuningTemplateMaker(xlsx_path,save_finetune_json )
ftm.make_template()

# RAG

In [9]:
vb_loader = TextVBLoader(transcripts_list_punct)
db = vb_loader.load_db()

/Users/ernestgatiatullin/Desktop/github/PyProjects/my-projects/DivertitoGPT/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Found existing index


In [10]:
question = "Что такое флирт?"
searchDocs = db.similarity_search(question) 
print(searchDocs[0].page_content)

question = "В чем заключается теория 100 баллов?"
retriever = db.as_retriever(search_kwargs = {"k":2})
rel_docs = retriever.get_relevant_documents(question)
rel_docs

флирта воспринимать. она сама даже где-то проявляет инициативу тут я давай, выходи, там, сходим, позавтракаем. она едет к парню в отельный номер. да, это немножко не тот случай, когда вы типа с девочкой именно в отель, конкретно на него, конкретно отель, это по сути вас съемная квартира, но по сути да, какая по большого шатра с девушкой, или тебе домой к тебе, домой в твою квартиру, там, где есть кровать и никто на вас не будет смотреть. ясно, что в этот момент все уже зависит от того, насколько парень сам знает и понимает, что он хочет, насколько он сам понимает, какие действия он был делать. готова ли она тот момент? потому что утром и конечно готова. но уверенности действий парня должно хватить за двоих. то есть, если сам парень где-то там на пол шишечки что-то сделает с какими-то девушками, это достаточно. они до девушки пришли к тебе, это только там взял за павлович в кофте расстегнулась. она начинает сама растекается, растягиваться до раздеваться, и тебя еще может быть рождение, 

[Document(page_content='все на самом деле не настолько случайным образом происходит. Да, есть различные элементы на которые ты повлиять не можешь, но в принципе структура соблазнения она очень четкая и понятная, что тебе нужно знать, что тебе нужно понимать. Я хочу рассказать на примере теории 100 баллов. автор, по моему, этой теории является Филип богачев. Ну, или как минимум впервые я наткнулся на такой материал в такой подаче, с таким названием, именно в его книжке. там по-моему это была русская модель. в кино соблазнение, еще 1, какой-то выпуск, идет речь о том, что когда парень задается вопросом: хорошо, вот, есть девушка, есть я, Что я должен сделать или что от меня требуется чтобы она меня захотела, что требуется, чтобы наша история с ней развивалась. для понимания, для того, чтобы не просто говорить какими-то такими словами бесформенными, да А морфинами, амортом. мы постараемся загнать максимально, загнать это в некую логическую, какую-то цепочку, теория 100 баллов, представим,

In [24]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory

import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# if not os.environ.get("LANGCHAIN_API_KEY"):
#     os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [25]:
ChatMessageHistory()

ChatMessageHistory(messages=[])

In [13]:
system_prompt = """Ты помогаешь отвечать на вопросы, касающиеся личных взаимотношений мужчины и женщины. Отвечай
                на вопросы на основе контекста. Отвечай коротко и понятно.
                \n\n
                Контекст:
                {context}
                """
prompt= ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("human", "{input}"),
    ]
)                                 
question_answer_chain = create_stuff_documents_chain(ya_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
rag_chain.invoke({"input": "Что такое теория 100 баллов?"})

{'input': 'Что такое теория 100 баллов?',
 'context': [Document(page_content='все на самом деле не настолько случайным образом происходит. Да, есть различные элементы на которые ты повлиять не можешь, но в принципе структура соблазнения она очень четкая и понятная, что тебе нужно знать, что тебе нужно понимать. Я хочу рассказать на примере теории 100 баллов. автор, по моему, этой теории является Филип богачев. Ну, или как минимум впервые я наткнулся на такой материал в такой подаче, с таким названием, именно в его книжке. там по-моему это была русская модель. в кино соблазнение, еще 1, какой-то выпуск, идет речь о том, что когда парень задается вопросом: хорошо, вот, есть девушка, есть я, Что я должен сделать или что от меня требуется чтобы она меня захотела, что требуется, чтобы наша история с ней развивалась. для понимания, для того, чтобы не просто говорить какими-то такими словами бесформенными, да А морфинами, амортом. мы постараемся загнать максимально, загнать это в некую логиче

In [17]:
contextualize_q_system_prompt = """ Сформулируй отдельный вопрос на основе истории чата и последнего вопроса
                                пользователя таким образом, чтобы этот вопрос был понятным без истории чата.
                                Не отвечай на вопрос, просто переформулируй, если нужно, или же оставь его как есть."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    ya_llm, retriever, contextualize_q_prompt
)     

history_aware_retriever.invoke({"input": "Кто ее изобрел?"})     

[Document(page_content='Всем привет С вами Максим диверситета и у нас сегодня рубрика история отношений. знаете какое название сам автор истории придумал? гей москва-мажоры, инфоцыгане, содержанка, что-то перемешано, так всё, кто-то в единственном количестве, кто-то во множественном. Я думаю его вдохновило то что на канале, Ну не так давно разбирал историю милфа, сказка криминал, и он такой думает о Макс. тогда повёлся на название, и у меня названии поведётся. В общем название я явно изменю, мне оно не нравится, а историю разберу мало того. Судя по объёму текста, скорее всего этот видос будет около часа. Надеюсь, что мне будет интересно снимать, Надеюсь, что вам будет интересно смотреть, как оно будет на самом деле в процессе разберёмся погнали Всем привет. немного обо мне. 27 и я могу сказать о себе две вещи. Задумайтесь, Если вы могли себе сказать две вещи, чтобы вы сказали. а вот автор говорит: первое: - я клёвый по характеру, Ну как и мы с вами. Да. второе: я внешний не сильно прив

In [19]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(ya_llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [20]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
question = "Что такое теория 100 баллов?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "Кто ее изобрел?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Как становится понятным из контекста, автором или, как минимум, тем, кто популяризировал теорию 100 баллов в изложении Филипа Богачёва, является он сам. Также сложно однозначно определить, является ли он именно изобретателем этой теории, поскольку подобие концепций, связанных с соблазнением, предлагалось и ранее, в том числе в кинематографе и литературе.


In [108]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

store = {}


def summarize_history(history: BaseChatMessageHistory, max_len):

    summary_template = """ Дай краткое содержание истории чата пользователя с виртуальным ассистентом.
                            Оно должно отвечать на вопрос "О чем мы говорили в предыдущих сообщениях?"
                            "История чата":
                            {text}
                            """
    summary_prompt= PromptTemplate(
        input_variables=["text"],
        template=summary_template
        
    ) 
    text = " ".join([chat.content for chat in store["abc123"].messages])
    summary_chain = LLMChain(llm=ya_llm, prompt=summary_prompt)    
    condensed_history = [
            HumanMessage(content="О чем мы говорили в предыдущих сообщениях"),
            AIMessage(content=summary_chain.invoke({"text" : text})["text"]),
        ]
    history.messages = condensed_history

def get_session_history(session_id: str, max_len=100) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    else:
        if len(store[session_id].messages) > max_len:
            summarize_history(store[session_id], max_len)
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)
conversational_rag_chain.invoke(
    {"input": "Что такое теория 100 баллов?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Теория 100 баллов — это концепция, описывающая параметры, которые определяют вероятность успешного соблазнения девушки. Согласно этой концепции, существует 100 условных баллов, и когда мужчина набирает их, девушка с большой вероятностью согласится на интимную близость. Автор этой теории — Филип Богачев. По его мнению, завоевать женщину можно флиртом и интересным общением, а также доброжелательностью и умением излагать мысли.'

In [110]:
conversational_rag_chain.invoke(
    {"input": "В чем она заключается?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Концепция 100 очков основывается на флирте, разговоре и эмоциях, которые должны быть включены на протяжении всего общения. Флирт, интересный разговор и проявление внимания могут заинтриговать женщину так, что она заинтересуется вами, оценит ваш интерес к ней. Важно также проявить  эмоциональную связь, показать заботу о собеседнице и уметь учитывать её состояния и реакции, так вы показываете заинтересованность и возможность более глубокой связи.\nЭтот подход может помочь в создании интимной обстановки, особенно в рамках романтического контекста. Однако следует учитывать, что эта концепция не абсолютна, и успех в соблазнении сильно зависит от вашей личности, отношения женщины к вам, а также от многих факторов, таких как ваша уверенность и умение всё это выразить. Не следует сводить весь процесс флирта и соблазнения исключительно к цифрам или теоретической оценке до наступления конкретной ситуации. Также стоит напомнить о необходимости уважать женщин и их чувства, избегать любых форм ман

In [113]:
store

{'abc123': ChatMessageHistory(messages=[HumanMessage(content='О чем мы говорили в предыдущих сообщениях'), AIMessage(content='В последнем сообщении шла речь о концепции 100 баллов — теории об условиях, необходимых для согласия девушки на интимную близость. Согласно этой концепции, 100 **«очков» можно набрать при флирте с девушкой и общении с ней.** По мнению её автора Филиппа Богачёва, **флирт, интересный диалог и хорошее отношение помогут расположить к себе девушку и заинтересовать её.**\n\nОднако стоит помнить, что концепция 100 очков не абсолютна. Соблазнение основано на многих факторах, среди которых личность человека, его отношение к девушке, её восприятие и настроение. Кроме того, важно сохранять уважение к женскому полу и избегать манипуляций и насильственных действий.'), HumanMessage(content='Кто ее изобрел?'), AIMessage(content='Концепцию 100 «очков» придумал Филипп Богачёв, основатель движения «Пикап. Изучение методов пикапа, иногда практиковавшееся в России в конце 90-х и на

In [104]:
store

{'abc123': ChatMessageHistory(messages=[HumanMessage(content='О чем мы говорили в предыдущих сообщениях'), AIMessage(content='В предыдущих сообщениях мы обсуждали «**Теорию 100 баллов**» — систему, предложенную автором Филиппом Богочевым, состоящую из 100 пунктов, определяющих критерии успешного сближения с девушкой. Ранее мы рассмотрели пять основных пунктов:\n\n* **Что вы говорите.** Девушки ценят объём информации и интересное, эмоциональное и живое общение. Не следует звучать скучно и формально.\n\n* **Как вы говорите.**  Не стоит говорить монотонно, важно проявлять свою энергичность и уверенность в себе.\n\n*  **Внешний вид.** Необходимо соблюдать основные правила нахождения в общественных местах, а также гигиены и аккуратности.   \n\n* Презентабельность включала рассмотрение окружения, соответствующий общему сценарию поведения, возможность дружелюбного и открытого общения.\n\nВ конце мы обсудили пункт **«История знакомства»**. Ознакомление с информацией об интересующей вас персоне

In [105]:
conversational_rag_chain.invoke(
    {"input": "Кто ее изобрел?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Изобретателем «Теории 100 баллов» является, вероятно, Филипп Богачёв. Однако это лишь возможная версия авторства, поскольку в исходном тексте автора теории не названо.'

In [98]:
store

{'abc123': ChatMessageHistory(messages=[HumanMessage(content='О чем мы говорили в предыдущих сообщениях'), AIMessage(content='Теория 100 баллов — это объяснение того, как работает процесс соблазнения и как можно понять, чего хочет от вас девушка, чтобы соблазнить её. Теория объясняет, что можно оценить шансы на успех с конкретной девушкой, исходя из неких условий, которые можно назвать в сумме 100 баллами.  \n\nВот как объясняет эту теорию контекст предоставленного вами текста: некоторые мужчины считают, что девушка «захочет его», когда они наберёт некое определённое количество «баллов». Эти баллы представляют собой **пять факторов**, на которые обращают внимание девушки:  \n1. **Что говорит:** степень подробности и эмоциональный окрас речи.  \n2. **Как говорит:** весёлый, скучный или отстранённый голос.  \n3. **Внешний вид:** заботитесь ли вы о своей внешности и стиле.  \n4. **Привязанность:** какое внимание вы уделяете девушке и как выражаете заботу о ней.  \n5. **Ресурсы и положение

In [31]:
conversational_rag_chain.invoke(
    {"input": "В чем она заключается?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Могу предложить следующую краткую интерпретацию теории 100 баллов: существует универсальная формула, по которой можно оценить успешность взаимоотношений мужчин и женщин, разбив эту оценку на баллы от 0 до 100. Составитель этой формулы предложил набор из пяти аспектов, на которые стоит обратить внимание, присвоив каждому из них определённое количество баллов.'

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain

# (PromptTemplate.from_template(template=template,partial_variables={"context":'context'}))
# help(PromptTemplate.from_template)

help(PromptTemplate.from_template)

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

_template = """
«Учитывая историю чата и дополнительный вопрос,
перефразируйте дополнительный вопрос, сделав его отдельным вопросом на языке оригинала.
История чата:
{chat_history}
Дополнительный вопрос: {question}
Отдельный вопрос: """

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
template = """Ответь на вопрос на основе предоставленного контекста:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

In [ ]:
from yandex_chain import YandexLLM
ya_llm = YandexLLM(iam_token=token,folder_id = "b1gti4ai7mdnk1aocko1", use_lite=False)

from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
# prompt = PromptTemplate.from_template(
#     "Дай краткое содержаниие этого текста: {context}"
# )
template = (
    """ Представь, что ты Максим Дивертито - известный блоггер и коуч. 
    Ты знаешь все в области пикапа, соблазнения и построения отношений.
    Ты даешь человеку действенные советы по построению отношений, по соблазнению девушек, правильному поведению на свиданиях и объясняешь, 
    как совершать правильные мужские поступки. Ты категорически против измен и неверности, считаешь, 
    что возвращают бывших девушек только долбоебы. 
    Многие тысячи мужчины благодарны за твои советы, потому что они изменили их жизни в лучшую сторону. 
    Ты можешь использовать мат и нецензурную лексику в своих ответах, отвечать саркастично, шутливо или даже грубо."
 
    Ответь на текущий вопрос пользователя с учетом истории чата и данного контекста.
    История чата: {chat_history}
    Данный контекст: {context}
    Текущий вопрос: {question}"""
)
prompt = PromptTemplate.from_template(template=template)

llm_chain = LLMChain(llm=ya_llm, prompt=prompt)
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
retriever = db.as_retriever(search_kwargs={'k': 2})

# This controls how the standalone question is generated.
# Should take `chat_history` and `question` as input variables.

memory = ConversationSummaryBufferMemory(
    llm = ya_llm,
    memory_key="chat_history",
#     chat_memory=chat_history,  # this is your persistence strategy subclass of `BaseChatMessageHistory`
    output_key="answer",
    return_messages=True
)

question_generator_chain = LLMChain(llm=ya_llm, prompt=prompt)
chain = ConversationalRetrievalChain(
    memory = memory,
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
)

chain({"question": "Кто такой Максим Дивертито?."})

In [ ]:
chain({"question": "Сколько ему лет?"} , context = 'Context')

In [ ]:
from yandex_chain import YandexLLM
ya_llm = YandexLLM(iam_token=token,folder_id = "b1gti4ai7mdnk1aocko1", use_lite=False)

from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "Контекст: {context}"
)
llm_chain = LLMChain(llm=ya_llm, prompt=prompt)
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
retriever = db.as_retriever(search_kwargs={'k': 2})

# This controls how the standalone question is generated.
# Should take `chat_history` and `question` as input variables.
template = (
    """ Представь, что ты Максим Дивертито - известный блоггер и коуч. 
    Ты знаешь все в области пикапа, соблазнения и построения отношений.
    Ответь на текущий вопрос пользователя с учетом истории чата и данного контекста.
    История чата: {chat_history}
    Текущий вопрос: {question}"""
)
memory = ConversationSummaryBufferMemory(
    llm = ya_llm,
    memory_key="chat_history",
#     chat_memory=chat_history,  # this is your persistence strategy subclass of `BaseChatMessageHistory`
    output_key="answer",
    return_messages=True
)

prompt = PromptTemplate.from_template(template)
question_generator_chain = LLMChain(llm=ya_llm, prompt=prompt)
chain = ConversationalRetrievalChain(
    memory = memory,
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
)

chain({"question": "Кто такой Максим Дивертито?"})

In [ ]:
memory

In [ ]:
#ya_llm = YandexLLM(iam_token=token,folder_id = "b1gti4ai7mdnk1aocko1", use_lite=False)


question = "Привет, Максим! Как дела?"
rel_docs = retriever.get_relevant_documents(question)
template = """
Представь, что ты Максим Дивертито - известный блоггер и коуч. Ты знаешь все в области пикапа, соблазнения и построения отношений. Ты даешь человеку действенные советы по построению отношений, по соблазнению девушек, правильному поведению на свиданиях и объясняешь, как совершать правильные мужские поступки. Ты категорически против измен и неверности, считаешь, что возвращают бывших девушек только долбоебы. Многие тысячи мужчины благодарны за твои советы, потому что они изменили их жизни в лучшую сторону. Ты можешь использовать мат и нецензурную лексику в своих ответах, отвечать саркастично, шутливо или даже грубо.
Дай ответ на запрос на основе предоставленного контекста.

Краткое содеражание предыдущего диалога:
{chat_history}
-----
Контекст:
-----
{context}
-----
Вопрос:
{question}
"""
prompt = PromptTemplate.from_template(template=template)
QA_CHAIN_PROMPT = langchain.prompts.PromptTemplate.from_template(template)


memory = ConversationSummaryBufferMemory(
    llm = ya_llm,
    memory_key="chat_history",
#     chat_memory=chat_history,  # this is your persistence strategy subclass of `BaseChatMessageHistory`
    output_key="answer",
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    ya_llm,
    memory = memory,
    retriever=db.as_retriever(search_kwargs={'k': 2}),
    condense_question_prompt = QA_CHAIN_PROMPT,
#     return_source_documents=True,
)
# qa_chain({'question':question})

In [ ]:
qa_chain({'question':question, "context":"yes"})

In [ ]:
questions = ["Что такое флирт?", "Как познакомиться с девушкой?"]
for question in questions:
    qa_chain.invoke({'question':question})
    print(qa_chain)


In [ ]:
question = "Расскажи трешовую историю отношений."
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
Представь, что ты Максим Дивертито - известный блоггер и коуч. Ты знаешь все в области пикапа, соблазнения и построения отношений. Ты даешь человеку действенные советы по построению отношений, по соблазнению девушек, правильному поведению на свиданиях и объясняешь, как совершать правильные мужские поступки. Ты категорически против измен и неверности, считаешь, что возвращают бывших девушек только долбоебы. Многие тысячи мужчины благодарны за твои советы, потому что они изменили их жизни в лучшую сторону. Ты можешь использовать мат и нецензурную лексику в своих ответах, отвечать саркастично, шутливо или даже грубо.
Контекст:
-----
{context}
-----
Вопрос:
{query}
"""

prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=ya_llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
retriever = db.as_retriever(search_kwargs = {"k":2})
res = retriever.get_relevant_documents(question)

chain.run(input_documents=res, query=question)

In [ ]:
chain

In [ ]:
question = "Что такое флирт?"
max_calls = 20
while True:
    try:
        result = qa_chain({"query": question})
        if "Я не знаю" in result["result"]:
            max_calls-=1
            if max_calls == 0:
                print("Max model calls have been reached") 
                break
            else:
                result = qa_chain({"query": question})
        
        break
    except KeyError:
        max_calls-=1
        if max_calls == 0:
            print("Max model calls have been reached")
            break
        else:
            pass
result["result"],max_calls

In [ ]:
llm("""Ты интеллектуальный помощник в области пикапа, соблазнения и построения отношений.
Ты можешь использовать мат и нецензурую лексику в своих ответах.
Твоя задача отвечать на вопросы в рамках информации из предоставленного текста, если задают вопрос на русском языке, то отвечать нужно тоже на русском языке.
Будь умен во время поиска ответа.Если поиск не дал ответа, переформулируй вопрос и попробуй снова.
Оцени результат поиска и используй его, чтобы помочь тебе со следующими поисками, если это потребуется.
Если вопрос сложный, разбей его на маленькие шаги поиска и найти ответ за несколько шагов.
1)Отвечай точно по информации из предоставленного текста, даже если тебя просят придумать.
2)Если внутри контекста ответа нет, отвечай "Я не знаю".
3)Отвечай настолько подробно и развернуто, насколько тебе позволяет имеющаяся информация.

Вопрос: Что такое флирт?
Твой ответ: """)